In [ ]:
import pandas as pd
from pandas import DataFrame
import dateutil.parser
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv("4200_C005_2019_03_03.tsv", sep=',', header=None)
df.columns = ["kunag", "matnr", "date", "quantity","price"]

In [ ]:
df.head()

In [ ]:
del df['price']

In [ ]:
# dataframe.shape 
shape = df.shape 
print(shape)
# df.kunag.nunique(dropna = True) 
x= pd.value_counts(df.kunag)
print(len(x))
y= pd.value_counts(df.matnr)
print(len(y))

In [ ]:
df1 = df[(df > 0).all(1)]

# dataframe.shape 
shape = df1.shape 
print(shape)
# df.kunag.nunique(dropna = True) 
x= pd.value_counts(df1.kunag)
print(len(x))
y= pd.value_counts(df1.matnr)
print(len(y))

In [ ]:
def n_series(input_df,kunag,matnr):
    #removing the negative value rows
    input_df = input_df[input_df["quantity"] >=0]    
    df = input_df.copy()
    
    #finding a single customer(kunag),material(matnr) combination on the basis of input provided
    n_df1 = df[(df["kunag"] == kunag) & (df["matnr"] == matnr)]
    
    #parsing the date column in a datetime format
    n_df1.date = n_df1.date.apply(lambda x : pd.to_datetime(x,format = '%Y%m%d', errors='ignore'))
    #sort date start to end
    n_df1 = n_df1.sort_values('date')
    n_df1.set_index('date',inplace=True)
    
    #sampling the data on weekly basis (index is set to date first in the above step to do weekly sampling
    weekly_resampled_data = n_df1.quantity.resample('W').sum() 
    weekly_resampled_data = weekly_resampled_data.replace(np.nan, 0)
    individual_series = weekly_resampled_data.to_frame() 
    
    #resetting index so that date can be used as column
#     individual_series = individual_series.reset_index()
    return individual_series

In [ ]:
import os, sys,time
start = time.time()
bucket = pd.read_csv("bucket.csv")
df = pd.read_csv("4200_C005_2019_03_03.tsv", sep=',', header=None)
df.columns = ["kunag", "matnr", "date", "quantity","price"]
k = bucket.shape[0]
cnt = 0
for i in range (1,k):
            kunag = int(bucket["kunag"].iloc[i])
            matnr = int(bucket["matnr"].iloc[i])
            index = str(kunag) +"_"+ str(matnr)
            print("count",cnt)
            print("index : ",index)
            individual_series = n_series(df,kunag,matnr)
#             print(individual_series)
            plt.figure(figsize=(16,8))
#             plt.plot(individual_series '.r-') 
            plt.plot( individual_series, marker='.', color='blue')
#             plt.plot(individual_series)
            plt.xlabel("time")
            plt.ylabel('quantity')
            plt.savefig("/home/rahul/Downloads/bharat/time_series1/high_freq_plots/" + 'Graph_{}.png'.format(index), format="PNG")  

            plt.show()

            plt.show()
            cnt+=1
            plt.close()
end = time.time()
print("Time Taken : ",end - start)